In [6]:
!pip install groq duckduckgo-search newspaper4k lxml_html_clean agno yfinance fredapi

In [7]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['PHI_API_KEY'] = userdata.get('PHI_API_KEY') #INPUT YOUR AGNO KEY (I had this key before the rebrand)

print("API keys have been set!")


API keys have been set!


# *Data Ingestion*

### ***Data ingestion*** happens through the tools integrated with the financial_intelligence_agent. Specifically, the agent uses the following tools to gather data:

###* YFinanceTools
###* DuckDuckGoTools
###* Newspaper4kTools

### When you provide a query to the financial_intelligence_agent, the agent determines the intent of the query and then uses the appropriate tools to fetch the necessary data to fulfill the request.

### The agent then processes and analyzes the ingested data based on its instructions and the detected intent to generate a response.



In [8]:
from textwrap import dedent
from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.yfinance import YFinanceTools
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.newspaper4k import Newspaper4kTools

# *Data Pipeline*

### The "pipeline" is more of a direct interaction:

### **User Query** -> **Agent** -> **Tool Selection** -> **Data Fetching** -> **Data Processing/Analysis** by Agent -> **Response**

### A Data Pipeline typically involve distinct stages like ***Data Extraction***, ***Cleaning***, ***Transformation***, and loading into a separate storage before analysis. In this case, the agent handles the data fetching and immediate processing within its workflow.

In [9]:
def classify_intent(query: str) -> str:
    """
    Classifies user intent into one of three categories:
    - market_analysis
    - research
    - credit_score
    """
    query_lower = query.lower()
    if any(word in query_lower for word in ["stock", "market", "price", "analysis", "pe ratio", "eps"]):
        return "market_analysis"
    elif any(word in query_lower for word in ["research", "trend", "future", "impact", "implication", "report"]):
        return "research"
    elif any(word in query_lower for word in ["credit", "risk", "default", "rating", "score", "worthiness"]):
        return "credit_score"
    else:
        return "general"

financial_intelligence_agent = Agent(
    model=Groq(id="llama3-70b-8192"),
    tools=[
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            stock_fundamentals=True,
            historical_prices=True,
            company_info=True,
            company_news=True,
        ),
        DuckDuckGoTools(),
        Newspaper4kTools(),
        # FredTools()
    ],
    description="A unified Financial Intelligence Agent handling analysis, research, and credit scoring.",
    instructions=dedent("""\
        You are the **Financial Intelligence Agent** – adapt based on intent classification.

        ## Role Mapping:
        - **Market Analysis** → Provide stock price, key metrics, industry trends, sentiment, and forward-looking outlook.
        - **Research** → Conduct investigative research using multiple sources, summarize, analyze trends, and predict implications.
        - **Credit Score** → Assess financial ratios, risks, management, assign creditworthiness score, explain methodology.

        ## Reporting Rules:
        - Always begin with a short Executive Summary.
        - Use tables for financial/quantitative data.
        - Assign creditworthiness score and its interpretation.
        - Justification for the assigned score and score partition.


    """),
    markdown=True,
    show_tool_calls=True,
    add_datetime_to_instructions=True,
)

# Example usage
user_query = "Evaluate Meta vs Google creditworthiness and provide a risk assessment"
intent = classify_intent(user_query)
print(f"Detected Intent: {intent}")

financial_intelligence_agent.print_response(user_query, stream=True)


Detected Intent: credit_score


Output()